In [ ]:
import os
import subprocess
import shutil
from pathlib import Path
import traceback
import argparse

### Main code

In [ ]:
class Bowtie2Aligner:
    def __init__(self, folder_path, file, processed_folder, subfolder):
        self.parent_path = Path(folder_path).parent
        self.contaminants_dir = self.parent_path/"contaminants.fa"
        self.bowtie2_index = self.parent_path/"contaminants_index"
        self.r1_filename = None
        self.r2_filename = None
        self.file_stem = file.name.split(".")[0]
        self.sam_output = processed_folder/subfolder/"samtools"/f"{self.file_stem}_mapped.sam" ## file_stem = og filename w/o both extensions
        self.rmcontam_output = processed_folder/subfolder/f"{self.file_stem}_unmapped.fastq.gz"
        self.contam_output = processed_folder/subfolder/f"{self.file_stem}_mapped.fastq.gz"

    def build_bowtie2_index(self):
        """
        Must have contaminants.fa in parent directory of folder_path.
            folder_path = Folder name that ends with 'processed_fastqs'
        """
        if not self.bowtie2_index.exists():
            try:
                cmd = ["bowtie2-build", "-f", 
                       str(self.contaminants_dir), 
                       str(self.bowtie2_index)]
                result = subprocess.run(cmd, 
                                        check = True, ## if command returns non-zero exit status, raise error
                                        capture_output = True, 
                                        text = True)
            except subprocess.CalledProcessError as e: ## error handling
                    print(f"Failed to build bowtie2 index: {e}")
                    print("STDERR:", e.stderr)
                    print("STDOUT:", e.stdout)
                    traceback.print_exc()
                    raise
        else:
            pass
        return result
        
    def single_reads(self, file):
        """
        Align single-end reads (merged/unpaired)
        """
        try:
            cmd = ["bowtie2", 
                    "-x", str(self.bowtie2_index), ## provides index that we created earlier
                    "-U", str(file), ## specifies single-read
                    "-S", str(self.sam_output), ## .sam file of contam reads
                    "--un-gz", str(self.rmcontam_output), ## merged/unpaired reads that failed to align (mrna)
                    "--al-gz", str(self.contam_output)] ## merged/unpaired reads that aligned ≥1 times
            result = subprocess.run(cmd, 
                                    check = True, 
                                    capture_output = True, 
                                    text = True)
        except subprocess.CalledProcessError as e: ## error handling
            print(f"Failed to align merged or unpaired fastq file {file.name}: {e}")
            print("STDERR:", e.stderr)
            print("STDOUT:", e.stdout)
            traceback.print_exc()
            raise
        return result

    def detect_reps(self, subfolder):
        """
        defines variables for r1 and r2 filenames
        """
        for r1_file in subfolder.glob("*unmerged_R1*"):
            self.r1_filename = r1_file
            self.r2_filename = r1_file.with_name(r1_file.name.replace("_R1_", "_R2_"))

    def paired_reads(self, file):
        """
        Align paired-end reads (unmerged)
        """
        try: 
            cmd = ["bowtie2",
                    "-x", str(self.bowtie2_index),
                    "-1", str(self.r1_filename),
                    "-2", str(self.r2_filename),
                    "-S", str(self.sam_output), ## .sam file of contam reads
                    "--un-conc-gz", str(self.rmcontam_output), ## unmerged reads that failed to align (mrna)
                    "--al-conc-gz", str(self.contam_output)] ## unmerged reads that aligned ≥1 times
            result = subprocess.run(cmd, 
                                    check = True, 
                                    capture_output = True, 
                                    text = True)
        except subprocess.CalledProcessError as e: ## error handling
            print(f"Failed to align unmerged fastq file {file.name}: {e}")
            print("STDERR:", e.stderr)
            print("STDOUT:", e.stdout)
            traceback.print_exc()
            raise
        return result
    
    def convert_sam():
        sam_filename = Path(self.sam_output).stem 
        bam_file = f"{sam_filename}.bam"

        try:
            subprocess.run(["samtools", "sort", "-O", "BAM", ## convert .sam to .bam and sort
                            "-o", str(bam_file), str(self.sam_output)],
                            check = True,
                            capture_output = True,
                            text = True)
            subprocess.run(["samtools", "index", str(bam_file)], ## create .bai from .bam
                            check = True,
                            capture_output = True,
                            text = True)
            subprocess.run(["rm", str(self.sam_output)], ## remove original .sam file
                            check = True,
                            capture_output = True,
                            text = True)
        except subprocess.CalledProcessError as e: ## error handling
            print(f"Failed to convert {sam_filename.name} to .bam file: {e}")
            print("STDERR:", e.stderr)
            print("STDOUT:", e.stdout)
            traceback.print_exc()
            raise

In [ ]:
def rmcontam_pipeline(folder_path, output_path):
    if not os.path.exists(output_path):
        os.makedirs(output_path) ## if output_path doesn't already exist, create new empty output folder

    input_dir = Path(folder_path) ## imported Path to improve readability of code below
    output_dir = Path(output_path)
    aligner = Bowtie2Aligner(input_dir) ## initialize class
    
    aligner.build_bowtie2_index() ## build bowtie2 index

    for subfolder in input_dir.iterdir(): ## amount of subfolders = number of replicates
        if subfolder.is_dir():
            processed_folder = output_dir/f"{subfolder.name}_bowtie2" ## processed folders for bowtie2 outputs
            processed_folder.mkdir(exist_ok=True) ## if directory already exists, suppress OSError

            for file in subfolder.glob("*.fastq.gz"): ## iterate through indiv. files in subfolder
                try:
                    ## run bowtie2 alignment functions
                    if "_merged" in file.name or "_unpaired" in file.name:
                        aligner.single_reads(file, processed_folder)
                    elif "_unmerged" in file.name:
                        aligner.paired_reads(file, processed_folder)
                    
                    ## run samtools function
                    aligner.convert_sam()
                    
                except Exception as e:
                    print(f"Failed to align {file.name} with bowtie2 and produce .bam files: {e}")
                    traceback.print_exc()
                    continue

In [ ]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser(description = "Run contaminant removal pipeline.")
    parser.add_argument("--input", required = True, help = "Input processed_fastqs folder name")
    parser.add_argument("--output", required = True, help = "Desired output folder name for processed files")
    parser.add_argument("-u", "--unzip", action = "store_true", help = "Unzips fastq files")

    args = parser.parse_args()

    print("Starting contaminant removal pipeline...")
    rmcontam_pipeline(args.input, args.output)
    print("Pipeline finished.")